# Как переводить

- Xcode / Editor / Export for Localization / (Repository-root)/CDDDA-loc.
- Translations.csv / Copy "en" column / Google translate from english to target language.
- Translations.csv / New column with header named after lang, like "ru" without quotes.
- Translations.csv / Paste appropriate translations to appropriate lines.
- l10n.ipynb (this notebook) / Run all cells, fix errors in CSV sanity check.
- l10n.ipynb (this notebook) / Check save log, that your language is present.
- Xcode / Editor / Import Localizations / Select your localization.
- Simulator / Change language of device to target one.
- Xcode / Run in simulator. Check that screen menu is appropriately localized.

In [147]:
import csv
import pathlib

import lxml.etree as etree

In [148]:
l10n_root = pathlib.Path('../CDDA-loc')
l10n_csv = pathlib.Path('../Translations.csv')
wrong_directory = l10n_root / 'CDDA-loc'

In [149]:
xliffs = list(l10n_root.glob('*/Localized Contents/*.xliff'))
xliffs


def csv_sanity_check():
    with open(l10n_csv) as f:
        reader = csv.reader(f)
        langs = next(reader)
        errors = 0
        line = 1

        for record in reader:
            line += 1
            source = record[0]
            if (len(source) == 1) or (source[1] == ' '):
                sym = source[0]

                for lang, txn in zip(langs, record):
                    if txn[0] != sym:
                        errors += 1
                        print(f'{line}: Symbol {sym} differs for language {lang}: {txn}')
                    if (len(source) == 1) and (len(txn) != 1):
                        errors += 1
                        print(f'{line}: Trailing symbols for {sym} in {txn}')
    return errors


def findall(tree, tag):
    return tree.findall(f'.//{{urn:oasis:names:tc:xliff:document:1.2}}{tag}')


def save_translation(xliff: pathlib.Path):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    with open(l10n_csv) as f:
        translations_csv_reader = csv.reader(f)
        langs = next(translations_csv_reader)[1:]
        translation_mappings = {row[0]: dict(zip(langs, row[1:])) for row in translations_csv_reader}
    
    lang = xliff.name.split('.')[0]
    trans_units = findall(tree, 'trans-unit')
    
    for trans_unit in trans_units:
        source = findall(trans_unit, 'source')[0]
        
        try:
            translation = translation_mappings[source.text][lang]
        except KeyError:
            print(f'txn for {lang}/{source.text} not found. Skipping.')
            continue

        try:
            target = findall(trans_unit, 'target')[0]
        except IndexError:
            target = etree.SubElement(trans_unit, 'target')

        target.text = translation
        
    tree.write(str(xliff), encoding='utf8')

    
def get_targets_from_xliff(xliff):
    with open(xliff) as f:
        tree = etree.parse(f)
    
    targets = (x.text for x in findall(tree, 'target'))
    return targets


assert not csv_sanity_check(), 'CSV file has errors'
assert not wrong_directory.is_dir(), f'Another localization directory {wrong_directory} found inside of the correct. To prevent mistakes, please delete both and then start from exporting from Xcode.'

In [150]:
for xliff in xliffs:
    if str(xliff).endswith('en.xliff'):
        continue
    print(f'Saving {xliff}')
    save_translation(xliff)

Saving ../CDDA-loc/pt.xcloc/Localized Contents/pt.xliff
txn for pt/N Toggle minimap not found. Skipping.
txn for pt/U Unload or empty wielded item not found. Skipping.
txn for pt/m View map not found. Skipping.
txn for pt/L Move view East not found. Skipping.
txn for pt/0 View help not found. Skipping.
txn for pt/: Center view not found. Skipping.
txn for pt/Y Manage zones not found. Skipping.
txn for pt/V List all items around the player not found. Skipping.
txn for pt/A Apply or use wielded item not found. Skipping.
txn for pt// Advanced inventory management not found. Skipping.
txn for pt/v View morale not found. Skipping.
txn for pt/{ Toggle map memory not found. Skipping.
txn for pt/a Apply or use item not found. Skipping.
txn for pt/& Craft items not found. Skipping.
txn for pt/_ Select martial arts style not found. Skipping.
txn for pt/} Toggle panel admin not found. Skipping.
txn for pt/P View message log not found. Skipping.
txn for pt/X Peek around corners not found. Skipping

In [151]:
print('\n'.join(get_targets_from_xliff([x for x in xliffs if str(x).endswith('en.xliff')][0])))

N Toggle minimap
1
U Unload or empty wielded item
m View map
Info
7
L Move view East
0 View help
: Center view
Look
4
D
R Read
Y Manage zones
Inventory
V List all items around the player
I
W
A Apply or use wielded item
/ Advanced inventory management
v View morale
{ Toggle map memory
Misc
6
3
9
SPACE
a Apply or use item
& Craft items
_ Select martial arts style
} Toggle panel admin
+ Re-layer armor/clothing
Combat
2
SPACE
P View message log
X Peek around corners
[ View/activate mutations
= Swap inventory letters
G Grab something nearby
K Move view North
( Disassemble items
e Examine nearby terrain
) View scores
c Close door
| Wait for several minutes
> Descend stairs
D Drop item to adjacent tile
- Recraft last recipe
TAB
" Movement mode menu
J Move view South
ESC
i Open inventory
M View missions
s Toggle burst/auto mode
m Change aim mode
* Toggle snap-to-target
f Fire wielded item
5
w Wield
r Reload item
< Ascend stairs
⮐
C
d Drop item
8
H Move view West
Craft
x Look around
@ View play

In [152]:
import gettext
import itertools
import os

import pandas as pd

locale_dir = pathlib.Path('../Libraries/Cataclysm-DDA/lang/mo')

gettext.bindtextdomain('cataclysm-dda', localedir=locale_dir)
gettext.textdomain('cataclysm-dda')

'cataclysm-dda'

In [153]:
def translate(text: str):
    for lang_dir in locale_dir.glob('./*'):
        if lang_dir.is_dir():
            language_code = lang_dir.name
            os.environ['LANGUAGE'] = language_code
            yield language_code, gettext.gettext(text)
            
print('\n'.join('\t'.join(x) for x in translate('Re-layer armor/clothing')))

pl	Zmień warstwy zbroi/ubrań
uk_UA	Re-layer armor/clothing
da	Re-layer armor/clothing
pt_BR	Alterar Camadas de armadura/roupa
ja	防具/衣類の階層を変更
el	Re-layer armor/clothing
it_IT	Re-layer armor/clothing
zh_TW	重新排序裝備層級
is	Re-layer armor/clothing
ru	Очерёдность брони/одежды
zh_CN	整理装束
sr	Re-layer armor/clothing
ar	Re-layer armor/clothing
hu	Páncélzat és ruha újrarétegezése
nl	Re-layer armor/clothing
es_AR	Reordenar armadura/ropa
nb	Re-layer armor/clothing
de	Bekleidungsschichten umsortieren
ko	보호구/의류 재정렬하기
id	Re-layer armor/clothing
fr	Organiser les armures
en	Re-layer armor/clothing
es_ES	Reordenar armadura/ropa
tr	Re-layer armor/clothing


In [160]:
def get_texts_from_csv():
    with open(l10n_csv) as f:
        reader = csv.reader(f)
        langs = next(reader)
        for record in reader:
            yield record[0]


def get_texts_for_translation():
    english_texts = get_texts_from_csv()
    
    for text in english_texts:
        if len(text) == 1:
            continue
        if text[1] == ' ':
            prefix, real_text = text[:2], text[2:]
        else:
            prefix = ''
            real_text = text
        for language, translation in translate(real_text):
            yield (text, language, prefix + translation)

        
df = pd.DataFrame.from_records(get_texts_for_translation(), columns='text language translation'.split())
df

,text,language,translation
0,N Toggle Minimap,pl,N Toggle Minimap
1,N Toggle Minimap,uk_UA,N Toggle Minimap
2,N Toggle Minimap,da,N Toggle Minimap
3,N Toggle Minimap,pt_BR,N Toggle Minimap
4,N Toggle Minimap,ja,N 切替/ミニマップ表示
...,...,...,...
2059,CDDA,id,CDDA
2060,CDDA,fr,CDDA
2061,CDDA,en,CDDA
2062,CDDA,es_ES,CDDA


In [161]:
df.describe()

,text,language,translation
count,2064,2064,2064
unique,84,24,773
top,SPACE,el,SPACE
freq,72,86,72


In [162]:
def analyze(df):
    for text, subframe in df.groupby(df.text):
        if subframe.describe().translation['unique'] == 1:
            status = 0
        else:
            status = 1

        yield text, status
        

status_df = pd.DataFrame.from_records(analyze(df), columns='text status'.split())
status_df

,text,status
0,! Toggle Safe Mode,1
1,""" Movement Mode Menu",1
2,# View Factions,1
3,$ Sleep,1
4,% Item Action Menu,1
...,...,...
79,w Wield,1
80,x Look Around,1
81,{ Toggle Map Memory,1
82,| Wait for Several Minutes,1


In [163]:
status_df.status.aggregate(bool).describe()

count       84
unique       2
top       True
freq        61
Name: status, dtype: object

In [164]:
status_df[status_df.status == 0]

,text,status
7,( Disassemble Items,0
10,* Toggle Snap-to-target,0
12,- Recraft Last Recipe,0
13,/ Advanced Inventory Management,0
22,BTAB,0
24,CDDA,0
25,Cataclysm RPG,0
30,ESC,0
31,F Toggle Attack Mode of Wielded Item,0
32,G Grab Something Nearby,0


In [190]:
problematic_lines = [
    'Toggle Snap-to-target',  # [%c] target self; [%c] toggle snap-to-target
    'Change Aim Mode',  # [%c] to switch aiming modes.
    'Toggle Burst/Auto Mode',  # [%c] to switch firing modes.
]

list(translate('Grab something nearby'))

[('pl', 'Chwyć coś w pobliżu'),
 ('uk_UA', 'Grab something nearby'),
 ('da', 'Grab something nearby'),
 ('pt_BR', 'Pegar algo próximo'),
 ('ja', '近くの物を掴む'),
 ('el', 'Grab something nearby'),
 ('it_IT', 'Grab something nearby'),
 ('zh_TW', '拖行鄰近的某物'),
 ('is', 'Grab something nearby'),
 ('ru', 'Схватить что-то рядом'),
 ('zh_CN', '抓住载具/家具'),
 ('sr', 'Grab something nearby'),
 ('ar', 'Grab something nearby'),
 ('hu', 'Ragadj meg valamit a közelben'),
 ('nl', 'Grab something nearby'),
 ('es_AR', 'Agarrar algo cercano'),
 ('nb', 'Grab something nearby'),
 ('de', 'Etwas in der Nähe ergreifen'),
 ('ko', '근처에 있는 것을 잡아끌기'),
 ('id', 'Grab something nearby'),
 ('fr', 'Saisir quelque chose à proximité'),
 ('en', 'Grab something nearby'),
 ('es_ES', 'Coger algo cercano'),
 ('tr', 'Grab something nearby')]